In [1]:
import json
import os

data_folder = "../data"
names_filepath = os.path.join(data_folder, "names.json")

names = None
with open(names_filepath, "r") as fp_:
    names = json.load(fp_)

len(names)
names = sorted(names)
names[:100]

names.index("Will Smith")
names[7958756]
names = names[7858756:7959756]

In [23]:
from datetime import datetime
from polyleven import levenshtein
from nltk import ngrams

def string_to_hist(str_):
    characters = "abcdefghijklmnopqrstuvxwyz"
    char_map = {
    }

    for ch in characters:
        char_map[ch] = 0
    for ch in str_:
        if ch in "abcdefghijklmnopqrstuvxwyz":
            char_map[ch] += 1
    return char_map

def string_to_vector(str_):
    weights = {
        "a": 1,
        "b": 3,
        "c": 3,
        "d": 3,
        "e": 2,
        "f": 3,
        "g": 3,
        "h": 5,
        "i": 2,
        "j": 4,
        "k": 4,
        "l": 4,
        "m": 4,
        "n": 5,
        "o": 1,
        "p": 4,
        "q": 6,
        "r": 4,
        "s": 3,
        "t": 4,
        "u": 2,
        "v": 5,
        "x": 8,
        "w": 7,
        "y": 9,
        "z": 10
    }
    array = []
    characters = "abcdefghijklmnopqrstuvxwyz"
    for i in range(len(characters)):
        array.append(0)
    hist = string_to_hist(str_)
    for i, ch in enumerate(characters):
        array[i] = hist[ch] * (i+1) * (i+1)
    return array

def get_partitions(sorted_sums, unique_sums):
    bounds = []
    for u in unique_sums:
        lower_bound = -1
        upper_bound = -1
        for i, t in enumerate(sorted_sums):
            if lower_bound == -1 and t[0] == u:
                lower_bound = i
            if lower_bound != -1 and t[0] != u:
                upper_bound = i
                break
        bounds.append((u, lower_bound, upper_bound))
    last_item = bounds[len(bounds) -1]
    bounds[len(bounds) - 1] = (last_item[0], last_item[1], len(sorted_sums) - 1)
    return bounds

def get_bounds_from_partitions(q, partitions, threshold=15):
    bounding_partitions = None
    for i, p in enumerate(partitions):
        if q >= p[0]:
            lower = i - threshold
            upper = i + threshold
            if lower < 0:
                lower = 0
            if upper > len(partitions) - 1:
                upper = len(partitions) - 1
            bounding_partitions = (partitions[lower], partitions[upper])

    if bounding_partitions is None:
         bounding_partitions = (partitions[len(partitions) - threshold - 1], partitions[len(partitions) - 1])
    
    lower_index = bounding_partitions[0][1]
    upper_index = bounding_partitions[1][1]
    return (lower_index, upper_index)


def compute_sums(dataset):
    vectors = []
    sums_ = []
    for name in dataset:
        vec = string_to_vector(name.lower())
        vectors.append(vec)
        sums_.append((sum(vec), name))

    return sorted(sums_)

def get_unique_sums(sorted_sums):
    just_nums = [x[0] for x in sorted_sums]
    return list(set(just_nums))

def preprocess(dataset):
    sums = compute_sums(dataset)
    unique_sums = get_unique_sums(sums)
    print("Unique sums ", len(unique_sums))
    partitions = get_partitions(sums, unique_sums)
    return sums, partitions


def query_in_sorted_sums(sentence, sorted_sums, partitions, threshold=15):
    start = datetime.now()

    max_tokens = 10
    combinations  = []
    for i in range(1, min(len(sentence), max_tokens)):
        gr = ngrams(sentence.split(), i + 1)
        for g in gr:
            combinations.append(" ".join(g))

    csums = [sum(string_to_vector(c.lower())) for c in combinations]
    same_sum = []
    min_distance = 999999999
    min_name = ""
    min_c = ""


    best_matches = []
    for csum, c in zip(csums, combinations):
        bounds = get_bounds_from_partitions(csum, partitions, threshold=threshold)
        partition = sorted_sums[bounds[0]:bounds[1]]
        for sum_, name in partition:
            distance = levenshtein(name, c, 3)
            if distance < min_distance:
                min_distance = distance
                min_name = name
                min_c = c
                best_matches.append((min_distance, min_name, min_c))

    end = datetime.now()
    print("Query for `{}`. Best match: `{}` (distance: `{}`, ngram: `{}`) - elapsed time: {} - threshold:{}".format(sentence, min_name, min_distance, min_c, end - start, threshold))


In [24]:
sums, partitions = preprocess(names)
print(len(sums), len(partitions))

Unique sums  4630
101000 4630


In [29]:
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=1000)
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=200)
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=100)
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=50)
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=30)
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=15)
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=12)
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=10)
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=5)
query_in_sorted_sums("I want to watch a movie with Will Smith", sums, partitions, threshold=2)

query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=1000)
query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=200)
query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=100)
query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=50)
query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=30)
query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=15)
query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=12)
query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=10)
query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=5)
query_in_sorted_sums("I want to watch a movie with Will Smit", sums, partitions, threshold=2)

Query for `I want to watch a movie with Will Smith`. Best match: `Will Smith` (distance: `0`, ngram: `Will Smith`) - elapsed time: 0:00:00.988646 - threshold:1000
Query for `I want to watch a movie with Will Smith`. Best match: `Will Smith` (distance: `0`, ngram: `Will Smith`) - elapsed time: 0:00:00.290822 - threshold:200
Query for `I want to watch a movie with Will Smith`. Best match: `Will Smith` (distance: `0`, ngram: `Will Smith`) - elapsed time: 0:00:00.165187 - threshold:100
Query for `I want to watch a movie with Will Smith`. Best match: `Will Smith` (distance: `0`, ngram: `Will Smith`) - elapsed time: 0:00:00.115940 - threshold:50
Query for `I want to watch a movie with Will Smith`. Best match: `Will Smith` (distance: `0`, ngram: `Will Smith`) - elapsed time: 0:00:00.088205 - threshold:30
Query for `I want to watch a movie with Will Smith`. Best match: `Will Smith` (distance: `0`, ngram: `Will Smith`) - elapsed time: 0:00:00.072059 - threshold:15
Query for `I want to watch a m